In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
# from firebase_admin import db

cred = credentials.Certificate("./serviceAccountKey.json")
default_app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [2]:
unitdata = db.collection('StaticData').stream()

In [3]:
listdata = []

for udata in unitdata:
    uid = udata.id
    udict = udata.to_dict()
    
    defect = udict['defective']
    batch_num = udict['batch_num']
    full_index = udict['full_index']
    full_position = udict['full_position']
    timestamp = udict['timestamp']
    
    listdata.append((uid, defect, full_index, full_position, batch_num, timestamp))

In [4]:
print(len(listdata))
print(listdata[0])

10000
('Unit1', False, 'Batch 0, Stack 0, Bucket 0, Row 0, Column 0__Batch 0, Rotator 0, Cylinder 0, Pipe 0, Height 0', 'low__low', 0, 1573345196)


In [5]:
defect_units = filter(lambda x: x[1] == True, listdata)

In [7]:
defect_units = list(defect_units)
len(defect_units)

107

In [8]:
stack_level = {}
rotator_level = {}

for du in defect_units:
    sl, rl = du[3].split("__")
    
    if sl not in stack_level:
        stack_level[sl] = 0
    stack_level[sl] += 1
    
    if rl not in rotator_level:
        rotator_level[rl] = 0
    rotator_level[rl] += 1
    
print(stack_level)
print(rotator_level)

{'high': 32, 'mid': 44, 'low': 31}
{'mid': 36, 'high': 41, 'low': 30}


In [9]:
stack_id_wise = {}
rotator_id_wise = {}

for du in defect_units:
    sid, rid = du[2].split("__")[0].split(", ")[1], du[2].split("__")[1].split(", ")[1]
    
    if sid not in stack_id_wise:
        stack_id_wise[sid] = 0
    stack_id_wise[sid] += 1
    
    if rid not in rotator_id_wise:
        rotator_id_wise[rid] = 0
    rotator_id_wise[rid] += 1
    
print(stack_id_wise)
print(rotator_id_wise)

{'Stack 3': 11, 'Stack 0': 12, 'Stack 5': 11, 'Stack 6': 11, 'Stack 7': 11, 'Stack 8': 6, 'Stack 9': 3, 'Stack 10': 9, 'Stack 11': 11, 'Stack 1': 8, 'Stack 2': 8, 'Stack 4': 6}
{'Rotator 1': 28, 'Rotator 0': 28, 'Rotator 2': 28, 'Rotator 3': 23}


In [10]:
batch_wise = {}

for du in defect_units:
    bid = du[2].split(", ")[0]
    
    if bid not in batch_wise:
        batch_wise[bid] = 0
    batch_wise[bid] += 1
    
print(batch_wise)

{'Batch 0': 32, 'Batch 1': 34, 'Batch 2': 38, 'Batch 3': 3}


In [24]:
import datetime
import time

hourwise = {}

for du in defect_units:
    ts = du[5]
    dt = datetime.datetime.fromtimestamp(ts)
    dthour = dt.hour
    
    if str(dthour) not in hourwise:
        hourwise[str(dthour)] = 0
    hourwise[str(dthour)] += 1
    
hourwise2 = []
for k, v in hourwise.items():
    hourwise2.append([int(k), v])
    
hourwise2 = list(sorted(hourwise2))
hourwise2

[[1, 12],
 [2, 17],
 [3, 3],
 [4, 15],
 [5, 17],
 [6, 2],
 [7, 15],
 [8, 17],
 [9, 6],
 [10, 3]]

In [17]:
stack_level_total = {}
rotator_level_total = {}

stack_id_wise_total = {}
rotator_id_wise_total = {}

batch_wise_total = {}
hourwise_total = {}

print(len(listdata))


for ld in listdata:
    sl, rl = ld[3].split("__")
    
    if sl not in stack_level_total:
        stack_level_total[sl] = 0
    stack_level_total[sl] += 1
    
    if rl not in rotator_level_total:
        rotator_level_total[rl] = 0
    rotator_level_total[rl] += 1
    

for ld in listdata:
    sid, rid = ld[2].split("__")[0].split(", ")[1], ld[2].split("__")[1].split(", ")[1]
    
    if sid not in stack_id_wise_total:
        stack_id_wise_total[sid] = 0
    stack_id_wise_total[sid] += 1
    
    if rid not in rotator_id_wise_total:
        rotator_id_wise_total[rid] = 0
    rotator_id_wise_total[rid] += 1
    
    
for ld in listdata:
    bid = ld[2].split(", ")[0]
    
    if bid not in batch_wise_total:
        batch_wise_total[bid] = 0
    batch_wise_total[bid] += 1
    
    
for ld in listdata:
    ts = ld[5]
    dt = datetime.datetime.fromtimestamp(ts)
    dthour = dt.hour
    
    if str(dthour) not in hourwise_total:
        hourwise_total[str(dthour)] = 0
    hourwise_total[str(dthour)] += 1
    
    
print(stack_level_total)
print(rotator_level_total)
print(stack_id_wise_total)
print(rotator_id_wise_total)
print(batch_wise_total)
print(hourwise_total)

10000
{'low': 3340, 'mid': 3330, 'high': 3330}
{'low': 3334, 'mid': 3333, 'high': 3333}
{'Stack 0': 1080, 'Stack 3': 810, 'Stack 1': 820, 'Stack 4': 810, 'Stack 5': 810, 'Stack 6': 810, 'Stack 7': 810, 'Stack 8': 810, 'Stack 9': 810, 'Stack 10': 810, 'Stack 11': 810, 'Stack 2': 810}
{'Rotator 0': 2710, 'Rotator 1': 2430, 'Rotator 2': 2430, 'Rotator 3': 2430}
{'Batch 0': 3240, 'Batch 3': 280, 'Batch 1': 3240, 'Batch 2': 3240}
{'1': 1350, '10': 280, '2': 1620, '3': 270, '4': 1350, '5': 1620, '6': 270, '7': 1350, '8': 1620, '9': 270}


In [25]:
hourwise_total2 = []
for k, v in hourwise_total.items():
    hourwise_total2.append([int(k), v])
    
hourwise_total2 = list(sorted(hourwise_total2))
hourwise_total2

[[1, 1350],
 [2, 1620],
 [3, 270],
 [4, 1350],
 [5, 1620],
 [6, 270],
 [7, 1350],
 [8, 1620],
 [9, 270],
 [10, 280]]

In [ ]:
level - **Bar Chart** <Stack / Rotator vs Count> 
Number - **Line Chart** Stack Wise / Rotator Wise  / 2x Units as well
Batch - **Pie Chart** / 2x Units as well
Hourly - **Line Chart** / 2x Units as well

In [46]:
chartdata = {
    "stackLevelChart_X": ["low", "mid", "high"],
    "stackLevelChart_Y": [stack_level["low"], stack_level["mid"], stack_level["high"]],
    "rotatorLevelCheck_X": ["low", "mid", "high"],
    "rotatorLevelCheck_Y": [rotator_level["low"], rotator_level["mid"], rotator_level["high"]],
    "batchPieChart_X": ["Batch 0", "Batch 1", "Batch 2", "Batch 3"],
    "batchPieChart_Y": [batch_wise["Batch 0"], batch_wise["Batch 1"], batch_wise["Batch 2"], batch_wise["Batch 3"]],
    "batchTotal_X": ["Batch 0", "Batch 1", "Batch 2", "Batch 3"],
    "batchTotal_Y": [batch_wise_total["Batch 0"], batch_wise_total["Batch 1"], batch_wise_total["Batch 2"], batch_wise_total["Batch 3"]],
    "HourlyLineChart_X": [x[0] for x in hourwise2],
    "HourlyLineChart_Y": [x[1] for x in hourwise2],
    "HourlyTotalChart_X": [x[0] for x in hourwise_total2],
    "HourlyTotalChart_Y": [x[1] for x in hourwise_total2]
}

In [47]:
chartdata

{'stackLevelChart_X': ['low', 'mid', 'high'],
 'stackLevelChart_Y': [31, 44, 32],
 'rotatorLevelCheck_X': ['low', 'mid', 'high'],
 'rotatorLevelCheck_Y': [30, 36, 41],
 'batchPieChart_X': ['Batch 0', 'Batch 1', 'Batch 2', 'Batch 3'],
 'batchPieChart_Y': [32, 34, 38, 3],
 'batchTotal_X': ['Batch 0', 'Batch 1', 'Batch 2', 'Batch 3'],
 'batchTotal_Y': [3240, 3240, 3240, 280],
 'HourlyLineChart_X': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'HourlyLineChart_Y': [12, 17, 3, 15, 17, 2, 15, 17, 6, 3],
 'HourlyTotalChart_X': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'HourlyTotalChart_Y': [1350,
  1620,
  270,
  1350,
  1620,
  270,
  1350,
  1620,
  270,
  280]}

In [49]:
doc_ref = db.collection("Stats").document("chartData")
doc_ref.set(chartdata)

update_time {
  seconds: 1573353955
  nanos: 190364000
}

In [51]:
db.collection("Stats").document("chartData").get().to_dict()

{'rotatorLevelCheck_X': ['low', 'mid', 'high'],
 'rotatorLevelCheck_Y': [30, 36, 41],
 'HourlyLineChart_X': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'HourlyTotalChart_X': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'HourlyTotalChart_Y': [1350,
  1620,
  270,
  1350,
  1620,
  270,
  1350,
  1620,
  270,
  280],
 'HourlyLineChart_Y': [12, 17, 3, 15, 17, 2, 15, 17, 6, 3],
 'batchPieChart_X': ['Batch 0', 'Batch 1', 'Batch 2', 'Batch 3'],
 'batchPieChart_Y': [32, 34, 38, 3],
 'batchTotal_X': ['Batch 0', 'Batch 1', 'Batch 2', 'Batch 3'],
 'batchTotal_Y': [3240, 3240, 3240, 280],
 'stackLevelChart_X': ['low', 'mid', 'high'],
 'stackLevelChart_Y': [31, 44, 32]}